# 逆ポーランド記法計算機

## アプリケーション仕様

逆ポーランド記法の数式を計算するアプリケーションを作成する

アプリケーション仕様は以下の通り

- 扱える演算子は `+` `-` `*` `/` `%` のみ
- 扱える数値は32ビット整数（`i32`）のみ

## 計算ロジックの実装

### 逆ポーランド記法の計算実行関数
- 入力:
    - 逆ポーランド記法で書かれた数式 (`&str`)
        - e.g. `1 2 + 3 4 + *`
- 出力:
    - 逆ポーランド記法で書かれた数式を計算した結果 (`i32`)
        - e.g. `21`

#### 逆ポーランド記法の処理
![01_RPN.drawio.png](./img/01_RPN.drawio.png)

In [2]:
// 文字列を空白区切りで分割
"1 2 + 4 +".split_whitespace()

SplitWhitespace { inner: Filter { iter: Split(SplitInternal { start: 0, end: 9, matcher: CharPredicateSearcher { haystack: "1 2 + 4 +", char_indices: CharIndices { front_offset: 0, iter: Chars(['1', ' ', '2', ' ', '+', ' ', '4', ' ', '+']) } }, allow_trailing_empty: true, finished: false }) } }

In [4]:
// 文字列を空白区切りで分割して Vec<&str> に変換
"1 2 + 4 +".split_whitespace().collect::<Vec<_>>()

["1", "2", "+", "4", "+"]

In [22]:
use std::collections::VecDeque; // 両端キューを使えるようにする

// トークン列を逆ポーランド構文木として処理
// VecDeque (両端キュー) は Vec と違い、頭からも尾からも取り出し・追加可能
fn eval(tokens: &mut VecDeque<&str>) {
    // スタック
    let mut stack = VecDeque::new();
    
    // トークン列の先頭から順にトークンを取り出していく
    while let Some(token) = tokens.pop_front() {
        if let Ok(x) = token.parse::<i32>() {
            // 対象トークンが数値である場合はスタックに乗せる
            stack.push_back(x);
        } else {
            // 対象トークンが数値以外の場合はスタックから数値を2つ取り出す
            let y = stack.pop_back().expect("invlaid syntax"); // 取り出せなかった場合は "invalid syntax" error
            let x = stack.pop_back().expect("invlaid syntax"); // 取り出せなかった場合は "invalid syntax" error
            
            // 対象トークンが演算子かどうか判定して計算実行
            let result = match token {
                "+" => x + y,
                "-" => x - y,
                "*" => x * y,
                "/" => x / y,
                "%" => x % y,
                _ => panic!("invalid token"), // 不明な文字列が含まれている場合は "invalid token" error
            };
            
            // 計算結果をスタックに乗せる
            stack.push_back(result);
        }
        // この時点のトークン列とスタックの状態を表示
        println!("tokens: {:?}, stack: {:?}", tokens, stack);
    }
}

let mut tokens = "1 2 + 4 +".split_whitespace().collect::<VecDeque<_>>();
eval(&mut tokens);

tokens: ["2", "+", "4", "+"], stack: [1]
tokens: ["+", "4", "+"], stack: [1, 2]
tokens: ["4", "+"], stack: [3]
tokens: ["+"], stack: [3, 4]
tokens: [], stack: [7]


概ね動作確認が出来たら、構造体にまとめて一つのプログラムとして完成させる

In [26]:
:dep evcxr_input

// => $ cargo add evcxr_input
//    evcxr_input: EvCxR Jupyter Kernel 上で標準入力を使えるようにするクレート

use std::collections::VecDeque;

/// RpnCalculator structure
/// .0: bool => verbose
struct RpnCalculator(bool);

impl RpnCalculator {
    /// constructor
    pub fn new(verbose: bool) -> Self {
        Self(verbose)
    }
    
    /// evaluate RPN formula
    pub fn eval(&self, formula: &str) -> i32 {
        let mut tokens = formula.split_whitespace().collect::<VecDeque<_>>();
        self.eval_main(&mut tokens)
    }
    
    /// @private evaluate RPN tokens
    fn eval_main(&self, tokens: &mut VecDeque<&str>) -> i32 {
        // スタック
        let mut stack = VecDeque::new();

        // トークン列の先頭から順にトークンを取り出していく
        while let Some(token) = tokens.pop_front() {
            if let Ok(x) = token.parse::<i32>() {
                // 対象トークンが数値である場合はスタックに乗せる
                stack.push_back(x);
            } else {
                // 対象トークンが数値以外の場合はスタックから数値を2つ取り出す
                let y = stack.pop_back().expect("invlaid syntax"); // 取り出せなかった場合は "invalid syntax" error
                let x = stack.pop_back().expect("invlaid syntax"); // 取り出せなかった場合は "invalid syntax" error

                // 対象トークンが演算子かどうか判定して計算実行
                let result = match token {
                    "+" => x + y,
                    "-" => x - y,
                    "*" => x * y,
                    "/" => x / y,
                    "%" => x % y,
                    _ => panic!("invalid token"), // 不明な文字列が含まれている場合は "invalid token" error
                };

                // 計算結果をスタックに乗せる
                stack.push_back(result);
            }
            // verbose モードであれば、この時点のトークン列とスタックの状態を表示
            if self.0 {
                println!("tokens: {:?}, stack: {:?}", tokens, stack);
            }
        }
        stack.pop_back().expect("invlaid syntax") // スタックに残っている数値 = 計算結果
    }
    
    /// execute REPL
    /// get formula from exvcr_input
    pub fn repl(&self) {
        loop {
            let formula = evcxr_input::get_string("(formula | quit)> ");
            
            if formula == "quit" {
                println!("see you...");
                break;
            }
            println!("{}", self.eval(&formula));
        }
    }
}

fn main() {
    let rpn = RpnCalculator::new(true); // verbose mode
    rpn.repl();
}
main();

(formula | quit)>  1 2 + 3 * 1 - 2 /


tokens: ["2", "+", "3", "*", "1", "-", "2", "/"], stack: [1]
tokens: ["+", "3", "*", "1", "-", "2", "/"], stack: [1, 2]
tokens: ["3", "*", "1", "-", "2", "/"], stack: [3]
tokens: ["*", "1", "-", "2", "/"], stack: [3, 3]
tokens: ["1", "-", "2", "/"], stack: [9]
tokens: ["-", "2", "/"], stack: [9, 1]
tokens: ["2", "/"], stack: [8]
tokens: ["/"], stack: [8, 2]
tokens: [], stack: [4]
4


(formula | quit)>  quit


see you...
